<a href="https://colab.research.google.com/github/dabin3178/KoBERT_Multi-label/blob/main/%EB%8B%A4%EB%B9%88_kobert_multi_label_%EC%97%90%EB%9F%AC%EC%88%98%EC%A0%95_hugging_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python --version

Python 3.10.11


In [2]:
# 설치 후 런타임 다시 시작 필요할 수도 (런타임 다시 시작 필요하면 실행창에 뜸)
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 14.5 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292701 sha256=ff542bdeb784b135aa743c6b73749530d271b73123ecc7727b5cd288a7340981
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, ht

In [3]:
# Hugging Face API로 KoBERT 다운로드
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-6wtfjoeo/kobert-tokenizer_f66a436cc0bf4bd9839966d5cb223eb7
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-6wtfjoeo/kobert-tokenizer_f66a436cc0bf4bd9839966d5cb223eb7
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=5e19bc847ddf610bf7e913b4632785bb4c3de522300a6fa4896b4f5311d42850
  Stored in directory: /tmp/pip-ephem-wheel-cache-krj_97lk/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [5]:
!pip3 install kobert-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
# ●
#from transformers import BertTokenizer

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
# GPU 사용 확인
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [8]:
# GPU 사용 시
device = torch.device("cuda:0")

In [9]:
# ★
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
# ●
#bert_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [10]:
# 파라미터 세팅
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [11]:
import pandas as pd

# 웰니스 데이터 시트별로 라벨링해 저장
wellness = pd.DataFrame(columns=['상담 데이터', 'label'])

for i in range(9):
  globals()['wellness{}'.format(i+1)] = pd.read_excel('/content/drive/MyDrive/LAMDA/웰니스 DSM-5 분류.xlsx', sheet_name = i)
  globals()['wellness{}'.format(i+1)]['label'] = i
  wellness = wellness.append(globals()['wellness{}'.format(i+1)][['상담 데이터', 'label']])

<ipython-input-11-843c516d101a>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wellness = wellness.append(globals()['wellness{}'.format(i+1)][['상담 데이터', 'label']])
<ipython-input-11-843c516d101a>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wellness = wellness.append(globals()['wellness{}'.format(i+1)][['상담 데이터', 'label']])
<ipython-input-11-843c516d101a>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wellness = wellness.append(globals()['wellness{}'.format(i+1)][['상담 데이터', 'label']])
<ipython-input-11-843c516d101a>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wellness = wellness.append(globals()['wellness{}'.format(i+1)][['

In [12]:
# 웰니스 데이터로 KoBERT 훈련을 위한 데이터 형태 만들기 

data_list = []

for q, label in zip(wellness['상담 데이터'], wellness['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [13]:
# 훈련, 테스트 데이터셋 분리 
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_len, pad=True, pair=False):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad = pad
        self.pair = pair
        self.dataset = dataset

    def __getitem__(self, idx):
        if self.pair:
            text = self.dataset[idx][0][0]
            text_pair = self.dataset[idx][0][1]
        else:
            text = self.dataset[idx][0]

        inputs = self.tokenizer.encode_plus(
            text,
            text_pair if self.pair else None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=self.pad,
            truncation=False
        )

        item = {
            'input_ids': torch.tensor(inputs['input_ids']),
            'attention_mask': torch.tensor(inputs['attention_mask']),
            'token_type_ids': torch.tensor(inputs['token_type_ids'])
        }

        item['labels'] = int(self.dataset[idx][1])

        # 패딩된 시퀀스/길이와 타입에 대한 내용/어텐션 마스크 시퀀스 세 가지 배열을 얻을 수 있도록 함
        padded_seq = torch.tensor(inputs['input_ids']).numpy()
        len_and_type = list(torch.tensor(inputs['token_type_ids']).shape) #dtype은 출력 안됨
        attention_mask_seq = torch.tensor(inputs['attention_mask']).numpy()

        return np.array(padded_seq), np.array(len_and_type), np.array(attention_mask_seq), item['labels']


    def __len__(self):
        return len(self.dataset)


# 데이터셋 생성
train_dataset = BERTDataset(dataset_train, tokenizer, max_len, True, True)
test_dataset = BERTDataset(dataset_test, tokenizer, max_len, True, True)

# 데이터로더 생성
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [15]:
train_dataset[0]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


(array([   2, 1409,    3, 4832,    3,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1]),
 array([64]),
 array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 3)

In [16]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [17]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [18]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [19]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_loader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [21]:
# 테스트용 데이터셋
test_df = pd.DataFrame({'articles':['우울해서 죽고싶어', '잠이 안 와', '요즘 자꾸 살쪄', '불안하고 심장이 두근거려', '약 먹은 것처럼 멍해']})
test_df

,articles
0,우울해서 죽고싶어
1,잠이 안 와
2,요즘 자꾸 살쪄
3,불안하고 심장이 두근거려
4,약 먹은 것처럼 멍해


In [22]:
# KoBERT 모델로 예측한 에브리타임 게시물 분류 결과 저장

import numpy as np

def softmax(X):
  exp_a = np.exp(X)
  sum_exp_a = np.sum(exp_a)
  y = exp_a / sum_exp_a   
  return y

def predict(df):

    for i in range(0, len(df)): # 게시글 총 문장 개수만큼 반복
      if i % 100 == 0:
        print(i)
      data = [df.iloc[i, 0], '0']
      print(data)
      dataset_another = [data]
      another_test = BERTDataset(dataset_another, tokenizer, max_len, True, True)
      test_dataloader = DataLoader(another_test, batch_size=batch_size, shuffle=False)
      
      model.eval()

      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)

          valid_length= valid_length
          label = label.long().to(device)

          out = model(token_ids, valid_length, segment_ids)

          for j in out:
            logits = j
            probability = torch.sigmoid(j).cpu().detach().numpy() # 시그모이드 함수 사용해 0과 1 사이값으로 변환
            #print(torch.sigmoid(j))

            # 시그모이드 변환값이 0.5이상이라면 라벨 할당
            temp = [] # 라벨 저장할 리스트
            for p in probability:
              if p > 0.5:
                temp.append(1)
              else:
                temp.append(0)
            print(temp)

            df.iloc[i, 1:] = temp
            
            # logits = logits.detach().cpu().numpy() # 라벨 예측 결과
            
            # df.iloc[i, 1] = np.argmax(logits)

In [23]:
predict(test_df)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


0
['우울해서 죽고싶어', '0']
[1, 1]


ValueError: ignored